In [15]:
import mlflow
# An object of Flask class is our WSGI application.
from flask import Flask, request
import pickle
import numpy as np
import pandas as pd
import json


In [13]:
uri = '/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/2/e6b8b8c4e9864995ac5be24d398a6ec4/artifacts/model'
final_model = mlflow.pyfunc.load_model(uri)

In [14]:
with open(f"/workspaces/mlops_zoomcamp_project/dumps/misc/preprocessed_data.pkl","rb") as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

In [21]:
df = pd.read_csv("/workspaces/mlops_zoomcamp_project/data/Student_performance_data.csv")

In [32]:


# load model
with open(f'/workspaces/mlops_zoomcamp_project/dumps/models/Support_Vector_Machine_model.pkl', 'rb') as f_in:
    model = pickle.load(f_in)


def prepare_features(df):

    if isinstance(df, dict):
        df = [df]
    if not isinstance(df, pd.DataFrame):    
        df = pd.DataFrame(df)

    # load misc pkl data
    with open(f'/workspaces/mlops_zoomcamp_project/dumps/misc/numeric_features.pkl', 'rb') as f_in:
        numeric_features = pickle.load(f_in)

    with open(f'/workspaces/mlops_zoomcamp_project/dumps/misc/categoric_features.pkl', 'rb') as f_in:
        categoric_features = pickle.load(f_in)

    # cleaning
    columns_to_drop = ['GPA', 'StudentID', 'Age', 'GradeClass']
    numeric_columns = [feat for feat in numeric_features if feat not in columns_to_drop]
    categoric_columns = [feat for feat in categoric_features if feat not in columns_to_drop]
    
    df[numeric_columns] = df[numeric_columns].astype('float64')

    
    # transforming
    for column in categoric_columns:  
        with open(f'/workspaces/mlops_zoomcamp_project/dumps/misc/{column}_label_encoder.pkl', 'rb') as f_in:
            encoder = pickle.load(f_in)
            df[column] = encoder.transform(df[column])
    
    for column in numeric_columns:
        with open(f'/workspaces/mlops_zoomcamp_project/dumps/misc/{column}_standard_scaler.pkl', 'rb') as f_in:
            scaler = pickle.load(f_in)
            df[column] = scaler.transform(df[[column]])

    features = df.to_dict(orient='records')
                          
    return features

In [29]:
prepare_features(pd.DataFrame([{
    'StudentID': 1001,
    'Age': 17,
    'Gender': 1,
    'Ethnicity': 0,
    'ParentalEducation': 2,
    'StudyTimeWeekly': 1.7803355189521382,
    'Absences': -0.8908223715622926,
    'Tutoring': 1,
    'ParentalSupport': 2,
    'Extracurricular': 0,
    'Sports': 0,
    'Music': 1,
    'Volunteering': 0,
    'GPA': 2.929195591667681,
    }]))

ValueError: If using all scalar values, you must pass an index

In [11]:
df = pd.read_csv("/workspaces/mlops_zoomcamp_project/data/Student_performance_data.csv")

In [12]:
import sys
sys.path.append('/workspaces/mlops_zoomcamp_project/')

from utils.preprocessing import *

In [18]:
def serve():
    data = request.get_json(force=True)

In [20]:
from flask import Flask, request

request.get_json(force=True)

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.

In [19]:
data = request.get_json(force=True) 

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.

In [13]:
prepare_features(df)

[{'StudentID': 1001,
  'Age': 17,
  'Gender': 1,
  'Ethnicity': 0,
  'ParentalEducation': 2,
  'StudyTimeWeekly': 1.7803355189521382,
  'Absences': -0.8908223715622926,
  'Tutoring': 1,
  'ParentalSupport': 2,
  'Extracurricular': 0,
  'Sports': 0,
  'Music': 1,
  'Volunteering': 0,
  'GPA': 2.929195591667681,
  'GradeClass': 2.0},
 {'StudentID': 1002,
  'Age': 18,
  'Gender': 0,
  'Ethnicity': 0,
  'ParentalEducation': 1,
  'StudyTimeWeekly': 0.9973762482121091,
  'Absences': -1.7176935833500317,
  'Tutoring': 0,
  'ParentalSupport': 1,
  'Extracurricular': 0,
  'Sports': 0,
  'Music': 0,
  'Volunteering': 0,
  'GPA': 3.042914833436377,
  'GradeClass': 1.0},
 {'StudentID': 1003,
  'Age': 15,
  'Gender': 0,
  'Ethnicity': 2,
  'ParentalEducation': 3,
  'StudyTimeWeekly': -0.9840451409833825,
  'Absences': 1.3535423461472853,
  'Tutoring': 0,
  'ParentalSupport': 2,
  'Extracurricular': 0,
  'Sports': 0,
  'Music': 0,
  'Volunteering': 0,
  'GPA': 0.1126022544661815,
  'GradeClass': 4.0

In [ ]:


# Flask constructor takes the name of 
# current module (__name__) as argument.
app = Flask(__name__)

model = pickle.load(open('model.pkl','rb'))

# The route() function of the Flask class is a decorator, 
# which tells the application which URL should call 
# the associated function.
@app.route('/predict',methods=['POST'])
def serve():
    data = request.get_json(force=True)
    prediction = model.predict([[data['age'],
                                data['hypertension'],
                                data['heart_disease'],
                                data['avg_glucose_level'],
                                data['bmi'],
                                data['Residence_type_Rural'],
                                data['Residence_type_Urban'],
                                data['work_type_Govt_job'],
                                data['work_type_Never_worked'],
                                data['work_type_Private'],
                                data['work_type_Self-employed'],
                                data['work_type_children'],
                                data['smoking_status_Unknown'],
                                data['smoking_status_formerly smoked'],
                                data['smoking_status_never smoked'],
                                data['smoking_status_smokes'],
                                data['ever_married_No'],
                                data['ever_married_Yes'],
                                data['gender_Female'],
                                data['gender_Male'],
                                data['gender_Other']]])
    
    #print(type(prediction[0]))
    print(prediction[0])

    output = prediction[0]
    return json.dumps(output, default=str)

# main driver function
if __name__ == '__main__':
    app.run()

In [22]:
import numpy as np
from flask import Flask, request, jsonify
import pickle
import mlflow

In [23]:
app = Flask(__name__)
uri = '/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/2/e6b8b8c4e9864995ac5be24d398a6ec4/artifacts/model'
model = mlflow.pyfunc.load_model(uri)

In [27]:
@app.route('/api',methods=['POST'])
def predict():
    data = request.get_json(force=True)
    prediction = model.predict([[np.array(data['exp'])]])
    output = prediction[0]
    return jsonify(output)

AssertionError: View function mapping is overwriting an existing endpoint function: predict

In [28]:
if __name__ == '__main__':
    app.run(port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/local/python/3.10.13/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/python/3.10.13/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/codespace/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/codespace/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/codespace/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/codespace/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/codespace/.venv/lib/python3.10/site-packages/ipykernel

SystemExit: 1